In [2]:
import numpy as np
import pandas as pd

<div class="alert alert-info"><center><h1>Биологический нейрон</h1></center></div>

**Нейрон** — электрически возбудимая клетка, которая предназначена для приёма извне, обработки, хранения, передачи и вывода вовне информации с помощью электрических и химических сигналов.  
  
Типичный нейрон состоит из тела клетки, дендритов и одного аксона:

![neuron](images/neuron.png)

Простыми словами можно описать принцип действия нейрона следующим образом:
- через дендриты в нейрон поступают сигналы (раздражители)
- Если комбинация сигналов превышает пороговый уровень - нейрон "выстреливает", т.е. передаёт сигнал дальше через аксон.

Нейроны могут соединяться один с другим, формируя нервные сети.

<div class="alert alert-info"><center><h1>Функции Активации (Activation Functions)</h1></center></div>

Простейшим механизмом активации является **Step activation**, когда перспептрон передаёт на выход значение только в том случае, если сумма взвешенных входящих сигналов больше заданного порога:

![step](images/step_activation.png)

При всей своей простоте, данная функция активации обладает критическим недостатком: она недифференцируемая.  Как результат, она не позволяет осуществлять процесс обучения персептрона.  
  
Для того, чтобы исправить это, было разработано множество других функций активаций, таких как:

![neuron](images/activation_functions.png)

<div class="alert alert-warning"><center><h1>Задание 1 (2 балла)</h1></center></div>

Напишите класс **ActivationFunction** и его подкласс **Sigmoid**, у которого будет функция `forward`, которая:
- будет принимать на вход число и будет сохранять его внутри объекта
- будет возвращать результат в соответствии с фукцией $\sigma(x) = \frac{1}{1 + e^{-x}}$

In [3]:
class ActivationFunction:
    
    def __init__(self) -> None:
        self.input = None
        self.output = None


class Sigmoid(ActivationFunction):

    def forward(self, value: np.ndarray) -> np.ndarray:
        self.input = value
        self.output = 1.0/(1.0+np.exp(-self.input))
        return self.output


In [4]:
sigmoid = Sigmoid()

In [5]:
sigmoid.forward(np.array([0.79277902, 0.99868032]))

array([0.68842773, 0.73079903])

---

---

<div class="alert alert-info"><center><h1>Персептрон</h1></center></div>

**Персептрон** -  математическая модель биологического нейрона, является базовым элементом нейронных сетей:

![neuron](images/perceptron.png)

**Персептрон** состоит из следующих ключевых элементов:
- `вход` - отвечает за получение входных значений. Является аналогом дендрита биологического нейрона
- `веса` - механизм "важности" входных значений. По аналогии с нейроном - это "толщина" дендрита
- `функция активации` - обрабатывает сумму взвешенных входных сигналов и передаёт результат на выход
- `выход` - отвечает за передачу итогового результата. Аналогичен аксону
  
Практически всегда к входным сигналам также добавляется "bias", который всегда = 1.  
Это позволяет не привязывать выход персептрона к 0 в случае, если все входные сигналы также равны 0 (как в механизме регрессии)

<div class="alert alert-warning"><center><h1>Задание 2 (4 балла)</h1></center></div>

напишите класс **Layer**, у когорого будут следующие входные параметры:
- **n_inputs** - количество входящих значений
- **n_outputs** - количество исходящих значений (в нашем случае = 1)
- **activation** - объект из семейства **ActivationFunction** (в нашем случае - **Sigmoid**)
  
При своём создании объект класса **Layer** должен также создавать атрибут `weights_`, в ктором будут рандомально инициализированны веса для входящих значений, а также для `bias`

Класс **Layer** должен иметь функцию `forward`, принимающую на вход массив *numpy*, и возвращающую результат функции активации (тоже в виде массива).  
Также эта функция должна сохранять полученные на вход значения внутри экземпляра

In [6]:
class Layer:

    def __init__(self, n_inputs=np.array([1]), n_outputs=np.array([1]), activation=Sigmoid()):
        
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.input = None
        self.z = None
        self.output = None
        self.activation = activation
        self.bias = np.random.normal()
        self.weights_ = np.random.normal(size=self.n_inputs)

    def forward(self, X: np.ndarray) -> np.ndarray:

        self.input = X
        self.z = []

        for example in self.input:
            self.z.append(example.dot(self.weights_.T) + self.bias)
        
        self.z = np.array(self.z)

        self.output = self.activation.forward(self.z)

        return self.output

In [7]:
layer = Layer(np.array([5]))

In [8]:
layer.forward(np.array([[1,2,3,4,5],
                        [6,7,8,9,10],]))

array([0.96978013, 0.99999941])

---

<div class="alert alert-warning"><center><h1>Задание 3 (2 балла)</h1></center></div>

напишите класс **LossFunction** и его подкласс **CrossEntropy**, у которого будет функция `loss`, которая будет принимать реальное бинарное значение *y_fact* и вероятность *y_prob* (оба параметра в виде np.array) и будет возвращать результат по формуле:  
  
$$
L = - \sum (y_{fact} * log(y_{prob}) + (1-y_{fact})*log(1-y_{prob}))
$$

In [9]:
class LossFunction:
    
    def __init__(self) -> None:
        self.y_fact = None
        self.y_prob = None
        self.error = None

class CrossEntropy(LossFunction):

    def loss(self, y_fact: np.ndarray, y_prob: np.ndarray) -> np.ndarray:
        
        self.y_fact = y_fact
        self.y_prob = y_prob
        self.error = np.array(-np.mean(self.y_fact * np.log(self.y_prob) + \
                        (1 - self.y_fact) * np.log(1 - self.y_prob)))

        return self.error

In [10]:
cross_entropy = CrossEntropy()

In [11]:
cross_entropy.loss(np.array([[0, 1]]), np.array([[0.68842773, 0.73079903]]))

array(0.73987037)

---

<div class="alert alert-info"><center><h1>Обучение. Forward and Backpropagation</h1></center></div>

Процесс обучения персептрона (и в целом нейросети) итеративен и состоит из следующих этапов:
- Сперва персептрон инициализируется с рандомальными весами
- Осуществляется цикл "вперёд":
  - Входные значения перемножаются с соответствующими весами и суммируются
  - Эта сумма подаётся на функцию активации
  - Функция активации возвращает итоговое значение
- Итоговое значение сравнивается с ожидаемым и высчитывается ошибка (Loss)
- Осуществляется цикл "назад":
  - при помощи `Chain Rule` рассчитываются частичные производные для всех элементов персептрона
  - исходя из заданного коэффициента обучения (`learning rate`, $\alpha$), веса $w_{i}$ корректируются
- Данный цикл повторяется заданное количество раз или до тех пор, пока итоговая ошибка не опустится ниже заданного порогового значения

![img](images/training.png)

### <center>Chain Rule</center>

Если нам дана функция $y=f(u)$, где $u = g(x)$, то тогда производная этой функции по $x$ будет равно:

$$
\frac{dy}{dx} = \frac{dy}{du}\frac{du}{dx}
$$

Тогда для того, чтобы понять, насколько изменение весов $w$ влияет на изменение $y$ (т.е. производные $\frac{dy}{dw_{i}}$), можно вычислить следующие производные:

![neuron](images/backpropagation.png)

<div class="alert alert-warning"><center><h1>Задание 4 (8 баллов)</h1></center></div>

Модифицируйте классы **Layer**, **LossFuncton** и **ActivationFunction** таким образом, чтобы можно было рассчитать их частичные производные, и добавьте функцию `back`, позволяющую осуществить backpropagation.

<div class="alert alert-danger"><center><h3>Это задание очень сложное, и даже частичное его выполнение будет учитываться</h3></center></div>

In [12]:
class CrossEntropy(LossFunction):

    def loss(self, y_fact: np.ndarray, y_prob: np.ndarray) -> np.ndarray:
        
        self.y_fact = y_fact
        self.y_prob = y_prob
        self.error = np.array(-np.mean(self.y_fact * np.log(self.y_prob) + \
                        (1 - self.y_fact) * np.log(1 - self.y_prob)))
        return self.error

    def prime(self) -> float:
        dLda = np.mean(((self.y_fact - 1)/(self.y_prob - 1) - self.y_fact/self.y_prob))
        return dLda

In [13]:
class Sigmoid(ActivationFunction):

    def forward(self, value: np.ndarray) -> np.ndarray:
        self.input = value
        self.output = 1.0/(1.0+np.exp(-self.input))
        return self.output

    def prime(self) -> float:
        dadz = np.mean(self.output * (1 - self.output))
        return dadz

In [14]:
class Layer:

    def __init__(self, n_inputs=np.array([1]), n_outputs=np.array([1]), activation=Sigmoid()):
        
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.input = None
        self.z = None
        self.output = None
        self.activation = activation
        self.bias = np.random.normal()
        self.weights_ = np.random.normal(size=self.n_inputs)

    def forward(self, X: np.ndarray) -> np.ndarray:

        self.input = X
        self.z = []

        for example in self.input:
            self.z.append(example.dot(self.weights_.T) + self.bias)
        
        self.z = np.array(self.z)

        self.output = self.activation.forward(self.z)

        return self.output
    
    def update_weights(self, learning_rate, dLda, dadz) -> None:

        dzdw = np.mean(self.input, axis=0)
        dzdb = 1

        self.weights_ -= learning_rate * dLda * dadz * dzdw
        self.bias -= learning_rate * dLda * dadz * dzdb

In [15]:
class Perceptron():

    def __init__(self,
                 n_inputs=np.array([1]),
                 activation=Sigmoid(),
                 lossfucn=CrossEntropy(),
                 lerning_rate=0.05
                 ) -> None:
        
        self.layer = Layer(n_inputs=n_inputs, activation=activation)
        self.lossfucn = lossfucn
        self.output = None
        self.learning_rate = lerning_rate
    
    def forward(self, X:np.ndarray) -> np.ndarray:
        self.input = X
        self.output = self.layer.forward(self.input)
        return self.output
    
    def loss_by_batch(self, y:np.ndarray) -> float:
        self.loss = self.lossfucn.loss(y, self.output)
        return self.loss
    
    def backward(self, y:np.ndarray) -> None:
        self.loss = self.loss_by_batch(y)
        self.dLda = self.lossfucn.prime()
        self.dadz = self.layer.activation.prime()
        self.layer.update_weights(self.learning_rate, self.dLda, self.dadz)
        return None

In [30]:
# Логческое И
df = pd.DataFrame(data={'x1': [1,1,0,0],
                        'x2': [1,0,1,0],
                        'y': [1,0,0,0],})
df

,x1,x2,y
0,1,1,1
1,1,0,0
2,0,1,0
3,0,0,0


In [50]:
perceptron = Perceptron(np.array([2]), lerning_rate = 0.01)

print(f'start weights: {perceptron.layer.weights_}')
print(f'start bias: {perceptron.layer.bias}')

for epoch in range(10000):

    # dff = df.sample(np.random.randint(1, 5), replace=True)
    # dff = df.sample(10, replace=True)
    # dff = df
    dff = df.iloc[0:4, :]
    X = dff.iloc[:, :-1].to_numpy()
    y = dff.iloc[:, -1:].to_numpy()

    perceptron.forward(X)
    perceptron.backward(y)

    loss = perceptron.loss
    if epoch % 100 == 0:
        print(f"Epoch: {epoch}, Loss: {loss}")

X = df.iloc[:, :-1].to_numpy()
y = df.iloc[:, -1:].to_numpy()

print(f'end predict {perceptron.forward(X)}')
perceptron.loss_by_batch(y)
print(f'end loss: {perceptron.loss}')
print(f'end weights: {perceptron.layer.weights_}')
print(f'end bias: {perceptron.layer.bias}')

start weights: [0.41788222 0.18226602]
start bias: -0.4425563783241034
Epoch: 0, Loss: 0.6665034100784957
Epoch: 100, Loss: 0.614819595386098
Epoch: 200, Loss: 0.5896827875779689
Epoch: 300, Loss: 0.5770719976728544
Epoch: 400, Loss: 0.5705626080262168
Epoch: 500, Loss: 0.5671236861771021
Epoch: 600, Loss: 0.5652737465577384
Epoch: 700, Loss: 0.564264710825426
Epoch: 800, Loss: 0.563708488466264
Epoch: 900, Loss: 0.5633993727433512
Epoch: 1000, Loss: 0.5632264907108242
Epoch: 1100, Loss: 0.5631293084405891
Epoch: 1200, Loss: 0.563074447139076
Epoch: 1300, Loss: 0.56304336079882
Epoch: 1400, Loss: 0.563025683800773
Epoch: 1500, Loss: 0.5630155956609101
Epoch: 1600, Loss: 0.563009815777907
Epoch: 1700, Loss: 0.5630064892633038
Epoch: 1800, Loss: 0.5630045643993835
Epoch: 1900, Loss: 0.5630034432711983
Epoch: 2000, Loss: 0.5630027850245423
Epoch: 2100, Loss: 0.5630023947625584
Epoch: 2200, Loss: 0.563002160655449
Epoch: 2300, Loss: 0.5630020182661635
Epoch: 2400, Loss: 0.5630019302733944


---

# <center>Удачи!</center>